# Age of Empires II Voobly Matches Data Exploration

## Load libraries and set options/environment

In [1]:
# Set notebook options
%load_ext autoreload
%autoreload 2

# libraries & Functions
import pyarrow.feather as feather
import pandas as pd
import warnings, sys, os 
from pandas.core.common import SettingWithCopyWarning
sys.path.append(os.path.dirname(os.getcwd()))

from utilities.utilities import *

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', 300)
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

logger.setLevel(logging.DEBUG)
%matplotlib inline

2022-06-30 22:03:00 | DEBUG     | (private) matplotlib data path: c:\Users\Parikshit_verma\AppData\Local\Programs\Python\Python39\lib\site-packages\matplotlib\mpl-data
2022-06-30 22:03:00 | DEBUG     | matplotlib data path: c:\Users\Parikshit_verma\AppData\Local\Programs\Python\Python39\lib\site-packages\matplotlib\mpl-data
2022-06-30 22:03:00 | DEBUG     | CONFIGDIR=C:\Users\Parikshit_verma\.matplotlib
2022-06-30 22:03:00 | DEBUG     | matplotlib version 3.3.2
2022-06-30 22:03:00 | DEBUG     | interactive is False
2022-06-30 22:03:00 | DEBUG     | platform is win32
2022-06-30 22:03:00 | DEBUG     | loaded modules: ['sys', 'builtins', '_frozen_importlib', '_imp', '_thread', '_warnings', '_weakref', '_io', 'marshal', 'nt', 'winreg', '_frozen_importlib_external', 'time', 'zipimport', '_codecs', 'codecs', 'encodings.aliases', 'encodings', 'encodings.utf_8', '_signal', 'encodings.latin_1', '_abc', 'abc', 'io', '__main__', '_stat', 'stat', '_collections_abc', 'genericpath', 'ntpath', 'os.pa

In [2]:
mysql_data = feather.read_feather('../../data/mysql_aoe2_matches.feather')

In [4]:
mongodb_data = pd.read_csv('../../data/mongodb_aoe2_matches.csv')

C:\Users\Parikshit_verma\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3012: DtypeWarning: Columns (22,23,45) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [8]:
import json

In [9]:
pd.DataFrame(json.loads(mongodb_data.players[0]))


,number,name,color,color_id,winner,rate_snapshot,rate_before,rate_after,mvp,human,score,military_score,economy_score,technology_score,society_score,units_lost,buildings_razed,buildings_lost,units_converted,food_collected,wood_collected,stone_collected,gold_collected,tribute_sent,tribute_received,trade_gold,relic_gold,feudal_time,castle_time,imperial_time,feudal_time_secs,castle_time_secs,imperial_time_secs,explored_percent,research_count,total_wonders,total_castles,total_relics,villager_high,research,civilization,timeseries,apm,units_trained,metrics,villager_allocation,transactions
0,2,[LAK]_BarTu_,Green,2,True,1631,1631,1646,False,True,3523,472,1761,1290,0,16,0,0,0,6151,5898,0,3240,0,0,0,0,0:10:57,0:23:36,None,657,1416.0,None,58,15,0,0,0,49,None,{'name': 'Incas'},None,None,None,None,None,[]
1,1,realking14,Yellow,3,False,1600,1600,1585,False,True,1865,282,993,590,0,41,0,0,0,4084,3281,240,400,0,0,0,0,0:11:46,None,None,706,NaN,None,45,4,0,0,0,29,None,{'name': 'Burmese'},None,None,None,None,None,[]


In [21]:
pd.DataFrame(json.loads(mongodb_data.files[0]))

,original_filename,size,language,encoding,owner,download_link
0,rec.20190330-170312.mgz,1704871,es,latin-1,{'name': '[LAK]_BarTu_'},/api/download/1


In [28]:
mongodb_data

,_id,added,build,cheats,dataset.name,difficulty,diplomacy_type,duration,event,files,game_version,has_playback,id,ladder.name,lock_teams,losing_team,map_name,map_reveal_choice,map_size,market,minimap_link,mirror,odds.civilizations,odds.teams,platform.match_url,platform.name,platform.url,platform_match_id,played,players,population_limit,postgame,rated,save_version,series,speed,team_size,teams,tournament,tribute,type,version,victory_condition,winning_team.players,winning_team.team_id,winning_team.winner
0,6136f52de279a1da6c592c37,2019-03-30 21:25:09.191743,NaN,False,Wololo Kingdoms,Easiest,1v1,0:30:28,NaN,"[{""original_filename"":""rec.20190330-170312.mgz"",""size"":1704871,""language"":""es"",""encoding"":""latin-1"",""owner"":{""name"":""[LAK]_BarTu_""},""download_link"":""/api/download/1""}]",VER 9.F,NaN,1,RM - 1v1,True,NaN,KotD2 - Arabia,Normal,Tiny,[],/api/map/1,False,"[{""wins"":1050,""losses"":853},{""wins"":853,""losses"":1050}]",NaN,https://www.voobly.com/match/view/,Voobly,https://www.voobly.com,19558641.0,2019-03-30 21:21:07.308338,"[{""number"":2,""name"":""[LAK]_BarTu_"",""color"":""Green"",""color_id"":2,""winner"":true,""rate_snapshot"":1631,""rate_before"":1631,""rate_after"":1646,""mvp"":false,""human"":true,""score"":3523,""military_score"":472,""economy_score"":1761,""technology_score"":1290,""society_score"":0,""units_lost"":16,""buildings_razed"":0,""b...",225,True,True,11.76,NaN,Standard,1v1,"[{""winner"":true,""players"":[{""name"":""[LAK]_BarTu_""}]},{""winner"":false,""players"":[{""name"":""realking14""}]}]",NaN,[],Random Map,Userpatch 1.5,Conquest,"[{""name"":""[LAK]_BarTu_""}]",1.0,True
1,6136f52de279a1da6c592c39,2019-03-30 21:25:11.204476,NaN,False,Wololo Kingdoms,Standard,1v1,0:26:25,NaN,"[{""original_filename"":""rec.20190330-220547.mgz"",""size"":1458734,""language"":""en"",""encoding"":""latin-1"",""owner"":{""name"":""MS_PAINT1""},""download_link"":""/api/download/2""}]",VER 9.F,NaN,2,RM - 1v1,True,NaN,Green Arabia,Normal,Tiny,[],/api/map/2,True,NaN,"[{""wins"":0,""losses"":3},{""wins"":3,""losses"":0}]",https://www.voobly.com/match/view/,Voobly,https://www.voobly.com,19558648.0,2019-03-30 21:23:07.022156,"[{""number"":1,""name"":""Doomheart"",""color"":""Yellow"",""color_id"":3,""winner"":false,""rate_snapshot"":1433,""rate_before"":1433,""rate_after"":1418,""mvp"":false,""human"":true,""score"":2098,""military_score"":96,""economy_score"":1172,""technology_score"":830,""society_score"":0,""units_lost"":7,""buildings_razed"":0,""build...",200,True,True,11.76,NaN,Standard,1v1,"[{""winner"":false,""players"":[{""name"":""Doomheart""}]},{""winner"":true,""players"":[{""name"":""MS_PAINT1""}]}]",NaN,[],Random Map,Userpatch 1.5,Conquest,"[{""name"":""MS_PAINT1""}]",1.0,True
2,6136f547e279a1da6c592c3b,2019-03-30 21:25:13.050197,NaN,False,Wololo Kingdoms,Standard,1v1,1:00:56,NaN,"[{""original_filename"":""rec.20190330-174451.mgz"",""size"":3458083,""language"":""en"",""encoding"":""latin-1"",""owner"":{""name"":""_TheSharp__""},""download_link"":""/api/download/3""}]",VER 9.F,NaN,3,RM - 1v1,True,NaN,Arabia,Normal,Tiny,[],/api/map/3,False,"[{""wins"":809,""losses"":1064},{""wins"":1064,""losses"":809}]","[{""wins"":2,""losses"":0},{""wins"":0,""losses"":2}]",https://www.voobly.com/match/view/,Voobly,https://www.voobly.com,19558637.0,2019-03-30 21:21:07.517456,"[{""number"":1,""name"":""_TheSharp__"",""color"":""Yellow"",""color_id"":3,""winner"":true,""rate_snapshot"":1909,""rate_before"":1909,""rate_after"":1921,""mvp"":false,""human"":true,""score"":15269,""military_score"":5264,""economy_score"":5117,""technology_score"":4238,""society_score"":650,""units_lost"":268,""buildings_razed""...",200,True,True,11.76,NaN,Standard,1v1,"[{""winner"":true,""players"":[{""name"":""_TheSharp__""}]},{""winner"":false,""players"":[{""name"":""Scanor""}]}]",NaN,[],Random Map,Userpatch 1.5,Conquest,"[{""name"":""_TheSharp__""}]",0.0,True
3,6136f668e1bc339bcfd9a9f8,2019-03-30 21:25:28.301286,NaN,False,Wololo Kingdoms,

In [85]:
pd.concat(list(temp.players.apply(lambda x: pd.json_normalize(json.loads(x)))))

,number,name,color,color_id,winner,rate_snapshot,rate_before,rate_after,mvp,human,score,military_score,economy_score,technology_score,society_score,units_lost,buildings_razed,buildings_lost,units_converted,food_collected,wood_collected,stone_collected,gold_collected,tribute_sent,tribute_received,trade_gold,relic_gold,feudal_time,castle_time,imperial_time,feudal_time_secs,castle_time_secs,imperial_time_secs,explored_percent,research_count,total_wonders,total_castles,total_relics,villager_high,research,timeseries,apm,units_trained,metrics,villager_allocation,transactions,civilization.name
0,1,Doomheart,Yellow,3,False,1433,1433,1418,False,True,2098,96,1172,830,0,7,0,1,0,4087,3543,220,1698,0,0,0,0,0:13:24,0:18:45,None,804,1125,None,33,11,0,0,0,35,None,None,None,None,None,None,[],Saracens
1,2,MS_PAINT1,Blue,0,True,1447,1447,1462,False,True,2615,84,1556,975,0,7,1,0,0,4769,4613,52,1800,0,0,0,0,0:10:57,0:22:53,None,657,1373,None,45,12,0,0,0,42,None,None,None,None,None,None,[],Saracens
0,1,_TheSharp__,Yellow,3,True,1909,1909,1921,False,True,15269,5264,5117,4238,650,268,13,2,0,27501,32782,4200,12554,0,0,0,619,0:10:12,0:25:27,0:41:08,612,1527,2468,97,46,0,5,2,136,None,None,None,None,None,None,[],Malay
1,2,Scanor,Blue,0,False,1817,1817,1805,False,True,10857,3807,3589,3201,260,355,3,14,0,23115,29149,1785,15506,0,0,0,933,0:11:07,0:23:34,0:44:58,667,1414,2698,79,34,0,2,2,138,None,None,None,None,None,None,[],Britons
0,1,_4LivesLeft,Green,2,True,1669,1669,1683,False,True,4087,617,1965,1505,0,41,7,0,0,10226,6563,0,3012,0,0,0,0,0:09:18,0:27:36,None,558,1656,None,68,18,0,0,0,57,None,None,None,None,None,None,[],Huns
1,2,theHand,Blue,0,False,1628,1628,1614,False,True,2420,711,1054,655,0,37,0,7,0,3795,4068,1050,321,0,0,0,0,0:09:10,None,None,550,NaN,None,44,5,0,0,0,36,None,None,None,None,None,None,[],Koreans
0,1,Limelight_,Green,2,True,1792,1792,1809,False,True,1072,135,457,480,0,2,1,0,0,1374,500,349,0,0,0,0,0,0:09:49,None,None,589,None,None,37,2,0,0,0,22,None,None,None,None,None,None,None,Huns
1,2,FAKE19MAIS,Gray,6,False,1814,1814,1797,False,True,863,16,467,380,0,10,0,1,0,1720,917,80,0,0,0,0,0,0:10:16,None,None,616,None,None,27,2,0,0,0,23,None,None,None,None,None,None,None,Portuguese
0,2,cristiangohe98,Orange,7,False,1524,1524,1506,False,True,3130,415,1590,995,130,32,1,0,0,5728,5089,1742,1975,0,0,0,0,0:11:06,0:24:07,None,666,1447,None,53,11,0,1,0,42,None,None,None,None,None,None,[],Byzantines
1,1,specialforce37,Blue,0,True,1473,1473,1491,False,True,3501,524,1631,1086,260,32,0,1,0,5519,5300,2015,1906,0,0,0,0,0:11:13,0:29:45,None,673,1785,None,56,10,0,2,0,42,None,None,None,None,None,None,[],Huns


In [81]:
temp['new'].explode()

KeyError: 0